# Camera LOW vs Video Frames

This notebook finds the most recent run (or uses a provided run dir), counts CAMERA_LOW events from serial telemetry, and compares that to the frame count in `raw.mp4` using `ffprobe`.


In [27]:
from __future__ import annotations

import csv
import os
import shutil
import subprocess
from pathlib import Path

try:
    import cv2
except Exception:
    cv2 = None

# Optional: set this to a specific run directory
run_dir: str | None = None  # e.g. '/home/jetson/Desktop/squeakview/runs/ds_2025-01-01_00-00-00_abcdef'

WORKSPACE = Path('/home/jetson/Desktop/squeakview')
RUNS_DIR = WORKSPACE / 'runs'
RUN_MARKER = RUNS_DIR / '.latest_run'

FFPROBE_PATH = (
    os.environ.get('FFPROBE_PATH')
    or shutil.which('ffprobe')
    or '/usr/bin/ffprobe'
)

def latest_run_dir() -> Path | None:
    if RUN_MARKER.exists():
        try:
            text = RUN_MARKER.read_text().strip()
            if text:
                path = Path(text)
                if path.exists():
                    return path
        except Exception:
            pass
    # Fallback: newest directory by mtime
    if not RUNS_DIR.exists():
        return None
    candidates = [p for p in RUNS_DIR.iterdir() if p.is_dir()]
    if not candidates:
        return None
    return max(candidates, key=lambda p: p.stat().st_mtime)

def count_camera_low(serial_csv: Path) -> int:
    count = 0
    with serial_csv.open(newline='') as f:
        reader = csv.reader(f)
        for row in reader:
            line = ','.join(row)
            if 'CAMERA_LOW' in line:
                count += 1
    return count

def _ffprobe_frame_count(video_path: Path) -> int:
    if not Path(FFPROBE_PATH).exists():
        return -1
    cmd = [
        FFPROBE_PATH,
        '-v', 'error',
        '-select_streams', 'v:0',
        '-count_frames',
        '-show_entries', 'stream=nb_read_frames',
        '-of', 'default=nokey=1:noprint_wrappers=1',
        str(video_path),
    ]
    out = subprocess.check_output(cmd, text=True).strip()
    if not out:
        return -1
    try:
        return int(out)
    except ValueError:
        return -1

def _opencv_frame_count(video_path: Path) -> int:
    if cv2 is None:
        raise FileNotFoundError('ffprobe not found and opencv-python is not installed')
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise FileNotFoundError(f'Cannot open video: {video_path}')
    count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if count > 0:
        cap.release()
        return count
    count = 0
    while True:
        ok, _ = cap.read()
        if not ok:
            break
        count += 1
    cap.release()
    return count

def get_frame_count(video_path: Path) -> int:
    count = _ffprobe_frame_count(video_path)
    if count >= 0:
        return count
    return _opencv_frame_count(video_path)

if run_dir is None:
    run_path = latest_run_dir()
else:
    run_path = Path(run_dir).expanduser()

run_path


PosixPath('/home/jetson/Desktop/squeakview/runs/D28241_2025-12-18_09-47-53')

In [28]:
if not run_path or not run_path.exists():
    raise FileNotFoundError('Run directory not found')

serial_csv = run_path / 'serial.csv'
video_path = run_path / 'raw.mp4'

if not serial_csv.exists():
    raise FileNotFoundError(f'Missing serial CSV: {serial_csv}')
if not video_path.exists():
    raise FileNotFoundError(f'Missing raw video: {video_path}')

def count_camera_low_after_start(serial_csv: Path) -> int:
    count = 0
    started = False
    with serial_csv.open(newline='') as f:
        reader = csv.reader(f)
        for row in reader:
            if not row:
                continue
            if row[0] == 'MARKER' and len(row) > 1 and row[1] == 'START_SENT':
                started = True
                continue
            if not started:
                continue
            line = ','.join(row)
            if 'CAMERA_LOW' in line:
                count += 1
    if started:
        return count
    # Fallback: no marker found, count all
    return count_camera_low(serial_csv)

camera_low_count = count_camera_low_after_start(serial_csv)
frame_count = get_frame_count(video_path)

print(f'Run dir: {run_path}')
print(f'CAMERA_LOW count: {camera_low_count}')
print(f'Frame count (raw.mp4): {frame_count}')
print(f'Match: {camera_low_count == frame_count}')


Run dir: /home/jetson/Desktop/squeakview/runs/D28241_2025-12-18_09-47-53
CAMERA_LOW count: 252193
Frame count (raw.mp4): 252193
Match: True


In [24]:
from collections import Counter
import csv

cap_csv = run_path / "capture_frames.csv"
counts = Counter()
last_ok_seq = 0

with cap_csv.open(newline="") as f:
    for row in csv.reader(f):
        if not row or row[0] == "ts_epoch_s":
            continue
        event = row[1]
        counts[event] += 1
        if event == "OK":
            last_ok_seq = int(row[2])

print("capture event counts:", counts)
print("OK count (from rows):", counts["OK"])
print("OK count (from last seq):", last_ok_seq)


capture event counts: Counter({'OK': 31266, 'START': 1})
OK count (from rows): 31266
OK count (from last seq): 31266


In [13]:
import csv

serial_csv = run_path / "serial.csv"
rows = []
with serial_csv.open(newline="") as f:
    reader = csv.reader(f)
    started = False
    for row in reader:
        if row and row[0] == "MARKER" and len(row) > 1 and row[1] == "START_SENT":
            started = True
            continue
        if not started:
            continue
        if "CAMERA_LOW" in ",".join(row):
            rows.append(row)

print("CAMERA_LOW rows after START:", len(rows))
print("First 5:", rows[:5])
print("Last 5:", rows[-5:])


CAMERA_LOW rows after START: 60672
First 5: [['CAMERA_LOW', '1764688424943187', '17424873375', 'nan', '1', '69420', '69420', '69420', 'Pellet_Available', 'nan'], ['CAMERA_LOW', '1764688424974690', '17424904878', 'nan', '2', '69420', '69420', '69420', 'Pellet_Available', 'nan'], ['CAMERA_LOW', '1764688425008415', '17424938603', 'nan', '3', '69420', '69420', '69420', 'Pellet_Available', 'nan'], ['CAMERA_LOW', '1764688425042388', '17424972577', 'nan', '4', '69420', '69420', '69420', 'Pellet_Available', 'nan'], ['CAMERA_LOW', '1764688425076412', '17425006600', 'nan', '5', '69420', '69420', '69420', 'Pellet_Available', 'nan']]
Last 5: [['CAMERA_LOW', '1764690447154934', '19447085122', 'nan', '60668', '69420', '69420', '69420', 'Pellet_Available', 'nan'], ['CAMERA_LOW', '1764690447188950', '19447119138', 'nan', '60669', '69420', '69420', '69420', 'Pellet_Available', 'nan'], ['CAMERA_LOW', '1764690447222884', '19447153072', 'nan', '60670', '69420', '69420', '69420', 'Pellet_Available', 'nan']

In [16]:
import csv
from collections import defaultdict

# Serial CAMERA_LOW timestamps (us) after START_SENT
ser_ts = []
started = False
with (run_path / "serial.csv").open(newline="") as f:
    reader = csv.reader(f)
    for row in reader:
        if not row:
            continue
        if row[0] == "MARKER" and len(row) > 1 and row[1] == "START_SENT":
            started = True
            continue
        if not started:
            continue
        if "CAMERA_LOW" in ",".join(row):
            try:
                ser_ts.append(int(row[1]))
            except Exception:
                pass

# Capture OK timestamps (s) → us
cap_ts = []
with (run_path / "capture_frames.csv").open(newline="") as f:
    reader = csv.reader(f)
    for row in reader:
        if not row or row[0] == "ts_epoch_s":
            continue
        ts_s, event, _seq = row
        if event == "OK":
            cap_ts.append(int(float(ts_s) * 1_000_000))

# Align to start time
ser0 = ser_ts[0]
cap0 = cap_ts[0]
ser_rel = [(t - ser0) // 1_000_000 for t in ser_ts]  # seconds since start
cap_rel = [(t - cap0) // 1_000_000 for t in cap_ts]

# Bucket by elapsed second
ser_counts = defaultdict(int)
cap_counts = defaultdict(int)
for s in ser_rel:
    ser_counts[int(s)] += 1
for s in cap_rel:
    cap_counts[int(s)] += 1

diffs = []
all_secs = sorted(set(ser_counts) | set(cap_counts))
for sec in all_secs:
    s = ser_counts.get(sec, 0)
    c = cap_counts.get(sec, 0)
    if s != c:
        diffs.append((sec, s, c, s - c))

print(f"Seconds with mismatch (elapsed time): {len(diffs)}")
print("First 10 mismatches (sec, serial, capture, diff):")
for row in diffs[:10]:
    print(row)


Seconds with mismatch (elapsed time): 49
First 10 mismatches (sec, serial, capture, diff):
(65, 30, 31, -1)
(69, 30, 29, 1)
(70, 30, 31, -1)
(148, 30, 31, -1)
(249, 30, 31, -1)
(332, 30, 31, -1)
(337, 30, 29, 1)
(338, 30, 31, -1)
(402, 30, 31, -1)
(417, 30, 29, 1)


In [17]:
import numpy as np

cap_ts = np.array(cap_ts, dtype=np.int64)
period_us = 33333  # expected at 30 fps
gaps = np.diff(cap_ts)

# Estimate missing frames: gap / period - 1
missing = np.maximum(0, np.round(gaps / period_us).astype(int) - 1)
print("Estimated missing frames from capture gaps:", int(missing.sum()))
print("Largest gap (ms):", gaps.max() / 1000.0)


Estimated missing frames from capture gaps: 0
Largest gap (ms): 48.856


In [18]:
import csv

# Capture OK timestamps (epoch us)
cap_ts = []
with (run_path / "capture_frames.csv").open(newline="") as f:
    reader = csv.reader(f)
    for row in reader:
        if not row or row[0] == "ts_epoch_s":
            continue
        ts_s, event, _seq = row
        if event == "OK":
            cap_ts.append(int(float(ts_s) * 1_000_000))

cap_start = cap_ts[0]
cap_end = cap_ts[-1]
cap_duration_us = cap_end - cap_start

# Serial TTL timestamps (us) after START_SENT
ser_ts = []
started = False
with (run_path / "serial.csv").open(newline="") as f:
    reader = csv.reader(f)
    for row in reader:
        if not row:
            continue
        if row[0] == "MARKER" and len(row) > 1 and row[1] == "START_SENT":
            started = True
            continue
        if not started:
            continue
        if "CAMERA_LOW" in ",".join(row):
            try:
                ser_ts.append(int(row[1]))
            except Exception:
                pass

ser0 = ser_ts[0]
ser_rel = [t - ser0 for t in ser_ts]

# Count serial TTLs that fall within capture duration
ser_in_window = sum(1 for t in ser_rel if 0 <= t <= cap_duration_us)

print("Capture OK count:", len(cap_ts))
print("Serial TTL in capture window:", ser_in_window)
print("Extra TTL outside capture window:", len(ser_ts) - ser_in_window)


Capture OK count: 60663
Serial TTL in capture window: 60640
Extra TTL outside capture window: 32


In [19]:
import csv

serial_csv = run_path / "serial.csv"

started = False
count = 0
first_idx = None
last_idx = None

with serial_csv.open(newline="") as f:
    reader = csv.reader(f)
    for row in reader:
        if not row:
            continue
        if row[0] == "MARKER" and len(row) > 1:
            if row[1] == "START_SENT":
                started = True
            elif row[1] == "STOP_SENT":
                break
            continue
        if not started:
            continue
        if "CAMERA_LOW" in ",".join(row):
            count += 1
            # 5th field looks like the Arduino counter (index)
            try:
                idx = int(row[4])
                if first_idx is None:
                    first_idx = idx
                last_idx = idx
            except Exception:
                pass

print("TTL count between START/STOP markers:", count)
if first_idx is not None and last_idx is not None:
    print("TTL count from Arduino index:", last_idx - first_idx + 1)


TTL count between START/STOP markers: 60672
TTL count from Arduino index: 60672


In [20]:
cap_csv = run_path / "capture_frames.csv"
ok_count = 0
with cap_csv.open(newline="") as f:
    reader = csv.reader(f)
    for row in reader:
        if not row or row[0] == "ts_epoch_s":
            continue
        if row[1] == "OK":
            ok_count += 1

print("Capture OK count:", ok_count)


Capture OK count: 60663


In [21]:
import csv
import numpy as np

# TTL intervals (us)
ser_ts = []
started = False
with (run_path / "serial.csv").open(newline="") as f:
    reader = csv.reader(f)
    for row in reader:
        if not row:
            continue
        if row[0] == "MARKER" and len(row) > 1 and row[1] == "START_SENT":
            started = True
            continue
        if not started:
            continue
        if "CAMERA_LOW" in ",".join(row):
            try:
                ser_ts.append(int(row[1]))
            except Exception:
                pass
ser_d = np.diff(np.array(ser_ts))
print("TTL interval (us) percentiles:",
      np.percentile(ser_d, [0, 1, 5, 50, 95, 99, 100]))

# Capture intervals (us)
cap_ts = []
with (run_path / "capture_frames.csv").open(newline="") as f:
    reader = csv.reader(f)
    for row in reader:
        if not row or row[0] == "ts_epoch_s":
            continue
        if row[1] == "OK":
            cap_ts.append(int(float(row[0]) * 1_000_000))
cap_d = np.diff(np.array(cap_ts))
print("Capture interval (us) percentiles:",
      np.percentile(cap_d, [0, 1, 5, 50, 95, 99, 100]))


TTL interval (us) percentiles: [31043. 31340. 31883. 33858. 34053. 34099. 35956.]
Capture interval (us) percentiles: [21219.   27852.22 29189.   33236.   37474.95 38901.17 48856.  ]
